In [1]:
#### label 정보
#### (배경 - 0 / 몸통 - 1 / 오른손 - 2 / 왼손 - 3 / 왼발 - 4 / 오른발 - 5 / 오른쪽 허벅지 - 6 / 왼쪽 허벅지 - 7 / 오른쪽 종아리 - 8 / 왼쪽 종아리 - 9 / 왼쪽 팔 - 10 / 
#### 오른쪽 팔 - 11 / 왼쪽 전완 - 12 / 오른쪽 전완 - 13 / 머리 - 14)
#### 해당 값이 RGB 값임 ex) 배경 0,0,0
### mask 확장자 png input image 확장자 jpg 해깔리지 말것

In [2]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)
import mmcv
import os.path as osp

1.9.0 True
0.18.0


In [3]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
## ver2용 추가
## 기존 mmsegmentation에 없거나 기능수정한 것 따로 data pipeline set
from custom_data_pipeline import CutOut,RandomAffine,CustomRandomFlip

In [4]:
classes = ('bg', 'body', 'right_hand', 'left_hand', 'left_leg', 'right_reg', 'right_thigh', 'left_thigh','right_calf','left_calf'
           ,'left_arm','right_arm','left_forearm','right_forearm','head')
## ver2용 추가 
flip_pair=[[2,3],[4,5],[6,7],[8,9],[10,11],[12,13]]  ##flip pair

In [5]:
palette = [[0,0,0], [1,1,1], [2,2,2], [3,3,3],[4,4,4], [5,5,5], [6,6,6], [7,7,7], [8,8,8], [9,9,9], [10,10,10], [11,11,11], [12,12,12], [13,13,13], [14,14,14]]

In [6]:
from mmcv import Config

In [7]:
from mmseg.apis import set_random_seed
## ver2용 model weight 저장시 100mb에 가장 근접하는 모델 사이즈로 선택..
cfg = Config.fromfile('mmsegmentation/configs/segformer/segformer_mit-b3_512x512_160k_ade20k.py')

In [8]:
cfg.dataset_type = 'body_seg'
cfg.data_root = '/root'


# Since we use ony one GPU, BN is used instead of SyncBN
## encoding 부분은 layer normalization 
## encoding 이후 ecoder part 부터는 batchnormalization
cfg.norm_cfg = dict(type='LN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = dict(type='BN', requires_grad=True)

# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 15

cfg.data.samples_per_gpu = 28
cfg.data.workers_per_gpu = 2

cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

cfg.crop_size = (256, 256)

### 수정 
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(256, 1024), ratio_range=(0.5, 2.0)),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **cfg.img_norm_cfg),
    ##### ver 2용 수정 #########
    # PIXEL VALUE와 무관한 변경이기 때문에 NORMALIZE 이후 적용 
    dict(type='CustomRandomFlip', prob=0.5, flip_pair=flip_pair), #기존 flip과 동일하게 horizontal flip 하면서 pair를 맞춰주는 거만 추가함 
    dict(type='RandomAffine',max_rotate_degree=20.0,
                             max_translate_ratio=0.3,
                             max_shear_degree=5.0),              ## 추가 (affine transform은 조금 강하게 줬을 때 학습 경향이 너무 안좋아져서 default로 사용하는 것으로..)
    dict(type='CutOut', n_holes=(0,6),cutout_shape=[(8,8),(8,16),(16,8),(16,16),(16,32),(32,16),(32,32)]),   ## 추가 (Cutout이 아닌 Random erase 형태로 적용함)
    ##### ver 2용 수정 #########
    dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(256, 1024),
        flip=False,
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5],
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]

cfg.data.train.type = cfg.dataset_type
cfg.data.train.data_root = cfg.data_root
cfg.data.train.img_dir = 'train2014'
cfg.data.train.ann_dir = 'train_mask'
cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.train.split = 'splits/train.txt'

cfg.data.val.type = cfg.dataset_type
cfg.data.val.data_root = cfg.data_root
cfg.data.val.img_dir = 'val2014'
cfg.data.val.ann_dir = 'val_mask'
cfg.data.val.pipeline = cfg.test_pipeline
cfg.data.val.split = 'splits/val.txt'


cfg.data.test.type = cfg.dataset_type
cfg.data.test.data_root = cfg.data_root
cfg.data.test.img_dir = 'val2014'
cfg.data.test.ann_dir = 'val_mask'
cfg.data.test.pipeline = cfg.test_pipeline
cfg.data.test.split = 'splits/val.txt'

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/train_segforemr_tuning_ver6'

##ver2 ADE20k로 pretrain 된 weight 사용 
cfg.load_from = 'work_dirs/train_segforemr_tuning_ver5/iter_141360.pth'

# 1epoch 대략 2600
cfg.runner.max_iters = 930*200
cfg.log_config.interval = 232
cfg.evaluation.interval = 930
cfg.checkpoint_config.interval = 930

# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


In [9]:
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset

@DATASETS.register_module()
class body_seg(CustomDataset):
  CLASSES = classes
  PALETTE = palette
  def __init__(self, split, **kwargs):
    super().__init__(img_suffix='.jpg', seg_map_suffix='.png', 
                     split=split, **kwargs)
    assert osp.exists(self.img_dir) and self.split is not None


In [10]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor


# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_segmentor(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES



# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True, 
                meta=dict())

2021-11-21 09:23:52,110 - mmseg - INFO - Loaded 26437 images
/opt/conda/lib/python3.7/site-packages/mmseg/models/backbones/mit.py:308: UserWarning: DeprecationWarning: pretrained is a deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is a deprecated, '
2021-11-21 09:23:54,423 - mmseg - INFO - Loaded 1908 images
2021-11-21 09:23:54,423 - mmseg - INFO - load checkpoint from work_dirs/train_segforemr_tuning_ver5/iter_141360.pth
2021-11-21 09:23:54,424 - mmseg - INFO - Use load_from_local loader
2021-11-21 09:23:54,748 - mmseg - INFO - Start running, host: root@67a5787c91ba, work_dir: /root/work_dirs/train_segforemr_tuning_ver6
2021-11-21 09:23:54,749 - mmseg - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
b

[                                                  ] 0/1908, elapsed: 0s, ETA:

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


[>>>>>>>>>>>>>>>>>>>>>>>>>>] 1908/1908, 13.3 task/s, elapsed: 143s, ETA:     0s

2021-11-21 09:42:05,417 - mmseg - INFO - per class results:
2021-11-21 09:42:05,419 - mmseg - INFO - 
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
|       bg      | 95.87 | 98.41 |
|      body     | 72.08 | 80.07 |
|   right_hand  | 58.71 |  75.3 |
|   left_hand   | 56.56 | 72.76 |
|    left_leg   | 37.17 | 45.17 |
|   right_reg   | 39.03 | 47.53 |
|  right_thigh  | 56.69 | 75.01 |
|   left_thigh  | 57.34 | 76.06 |
|   right_calf  | 56.98 | 73.71 |
|   left_calf   | 55.07 |  71.7 |
|    left_arm   | 63.73 | 76.46 |
|   right_arm   | 63.97 |  76.4 |
|  left_forearm | 64.67 | 79.84 |
| right_forearm | 65.05 | 79.93 |
|      head     | 66.48 | 74.87 |
+---------------+-------+-------+
2021-11-21 09:42:05,419 - mmseg - INFO - Summary:
2021-11-21 09:42:05,419 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 93.85 | 60.63 | 73.55 |
+-------+-------+-------+
2021-11-21 09:42:05,428 - mmse

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 1908/1908, 13.3 task/s, elapsed: 144s, ETA:     0s

2021-11-21 10:00:17,527 - mmseg - INFO - per class results:
2021-11-21 10:00:17,529 - mmseg - INFO - 
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
|       bg      | 95.91 | 98.28 |
|      body     | 72.22 | 81.23 |
|   right_hand  |  58.2 | 75.73 |
|   left_hand   | 56.45 | 72.88 |
|    left_leg   | 38.33 | 47.42 |
|   right_reg   | 39.22 | 47.89 |
|  right_thigh  | 56.08 |  75.4 |
|   left_thigh  | 57.04 | 75.92 |
|   right_calf  | 56.28 |  72.5 |
|   left_calf   | 53.86 | 69.46 |
|    left_arm   | 63.82 | 77.18 |
|   right_arm   | 63.42 | 75.85 |
|  left_forearm | 64.41 | 78.37 |
| right_forearm | 64.48 | 79.06 |
|      head     | 66.96 | 75.72 |
+---------------+-------+-------+
2021-11-21 10:00:17,529 - mmseg - INFO - Summary:
2021-11-21 10:00:17,530 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 93.83 | 60.44 | 73.52 |
+-------+-------+-------+
2021-11-21 10:00:17,533 - mmse

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 1908/1908, 13.2 task/s, elapsed: 144s, ETA:     0s

2021-11-21 10:18:29,894 - mmseg - INFO - per class results:
2021-11-21 10:18:29,896 - mmseg - INFO - 
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
|       bg      | 95.87 | 98.44 |
|      body     |  72.1 | 80.14 |
|   right_hand  | 58.43 | 76.31 |
|   left_hand   | 56.42 |  72.3 |
|    left_leg   | 36.04 | 43.92 |
|   right_reg   | 38.43 | 47.61 |
|  right_thigh  | 56.11 | 74.41 |
|   left_thigh  | 56.77 | 74.99 |
|   right_calf  | 55.42 |  71.4 |
|   left_calf   | 53.42 | 69.84 |
|    left_arm   | 63.66 | 76.56 |
|   right_arm   | 63.86 | 76.12 |
|  left_forearm | 64.43 | 78.62 |
| right_forearm | 64.68 | 80.17 |
|      head     | 66.81 | 75.15 |
+---------------+-------+-------+
2021-11-21 10:18:29,896 - mmseg - INFO - Summary:
2021-11-21 10:18:29,897 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 93.81 | 60.16 | 73.07 |
+-------+-------+-------+
2021-11-21 10:18:29,900 - mmse

KeyboardInterrupt: 